In [28]:
import json
import requests

from time import time
from pathlib import Path
from getpass import getpass

# Fetch Repices from KptnCook

In [24]:
api_key = getpass()
time_str = str(time())
today_url = f"https://mobile.kptncook.com/recipes/de/{time_str}?kptnkey={api_key}"

 ···························


In [25]:
r = requests.get(today_url)
r.raise_for_status()
recipes = r.json()

# Parse Recipes

In [4]:
# recipe_id = "5d30670b530000b23bc3a251"
# single_receipt_url = f"https://mobile.kptncook.com/recipes/{recipe_id}/details?kptnkey={api_key}"
# r = requests.get(single_receipt_url)
# r.raise_for_status()
# recipe = r.json()

In [5]:
from typing import Optional, List

from pydantic import BaseModel
from fastapi_camelcase import CamelModel

from mealie.schema.recipe.recipe import Recipe
from mealie.schema.recipe.recipe_step import RecipeStep
from mealie.schema.recipe.recipe_notes import RecipeNote
from mealie.schema.recipe.recipe_nutrition import Nutrition
from mealie.schema.recipe.recipe_ingredient import RecipeIngredient

In [31]:
class RecipeWithImage(Recipe):
    image_url: str


class LocalizedString(BaseModel):
    en: Optional[str]
    de: Optional[str]
    es: Optional[str]
    fr: Optional[str]
    pt: Optional[str]


class KCNutrition(BaseModel):
    calories: int
    protein: int
    fat: int
    carbohydrate: int


class KCImage(BaseModel):
    name: str
    type: Optional[str]
    url: str


class KptnCook(CamelModel):
    localized_title: LocalizedString
    country: str
    author_comment: LocalizedString
    preparation_time: int
    cooking_time: int
    recipe_nutrition: KCNutrition
    steps_en: List[str]
    steps_de: List[str]
    image_list: List[KCImage]
    
    def get_image_url(self, api_key):
        [image] = [i for i in self.image_list if i.type == "cover"]
        image_url = f"{image.url}?kptnkey={api_key}"
        return image_url
    

def transform_keys(raw):
    result = {}
    mapping = {"stepsEN": "steps_en", "stepsDE": "steps_de"}
    for k, v in raw.items():
        result[mapping.get(k, k)] = raw[k]
    return result


def kptncook_to_mealie(api: KptnCook):
    kwargs = {
        "name": api.localized_title.de,
        "notes": [
            RecipeNote(title="author comment", text=api.author_comment.de),
        ],
        "nutrition": Nutrition(
            calories=api.recipe_nutrition.calories,
            protein_content=api.recipe_nutrition.protein,
            fat_content=api.recipe_nutrition.fat,
            carbohydrate_content=api.recipe_nutrition.carbohydrate,
        ),
        "prep_time": api.preparation_time,
        "cook_time": api.cooking_time,
    }
    return Recipe(**kwargs)

In [33]:
recipe_api = KptnCook(**transform_keys(recipes[1]))
image_url = recipe_api.get_image_url(api_key)
recipe = kptncook_to_mealie(recipe_api)
recipe = RecipeWithImage(**recipe.dict(), image_url=image_url)

In [34]:
with Path("recipe.json").open("w") as f:
    f.write(recipe.json())

# Get Token

In [9]:
from getpass import getpass

token_url = "http://localhost:9000/api/auth/token"
login_data = {"username": "admin", "password": getpass()}
r = requests.post(token_url, data=login_data)
r.raise_for_status()
access_token = r.json()["access_token"]
headers={"authorization": f"Bearer {access_token}"}

 ········


# Build Client

In [10]:
class ClientOld:
    def __init__(self, base_url, headers):
        self.base_url = base_url
        self.headers = headers
        
    def to_url(self, path):
        return f"{self.base_url}{path}"
            
    def get(self, path, **kwargs):
        kwargs["headers"] = self.headers
        return requests.get(self.to_url(path), **kwargs)

    def post(self, path, **kwargs):
        kwargs["headers"] = self.headers
        return requests.post(self.to_url(path), **kwargs)

    def put(self, path, **kwargs):
        kwargs["headers"] = self.headers
        return requests.put(self.to_url(path), **kwargs)

In [11]:
class Client:
    """
    Proxy for requests, joining base_url with path and
    providing authentication headers automatically.
    """
    def __init__(self, base_url, headers):
        self.base_url = base_url
        self.headers = headers

    def to_url(self, path):
        return f"{self.base_url}{path}"
    
    def __getattr__(self, name):
        def inner(path, **kwargs):
            url = self.to_url(path)
            kwargs["headers"] = self.headers
            return getattr(requests, name)(url, **kwargs)
        return inner

In [12]:
client = Client("http://localhost:9000/api", headers)

In [13]:
self_path = "/users/self"
r = client.get(self_path)
r.raise_for_status()
self_data = r.json()
user_id, group_id = self_data["id"], self_data["groupId"]
print(user_id, group_id)

899979e8-f240-4d53-8ed3-eaeac21ecd53 71aae446-1aa5-4db7-9782-4021692cd0ce


# Post Recipe

In [14]:
import json

In [15]:
r = client.post("/recipes", data=recipe.json())
r.raise_for_status()
slug = r.json()
print("slug: ", slug)

slug:  high-protein-herzhaftes-waffelsandwich-mit-thunfisch-creme


In [16]:
recipe.json()

'{"id": null, "user_id": "642eb639-b76f-46fb-8ace-649c3766615e", "group_id": "dd9c980b-154c-4f59-882f-f15a146b5ab7", "name": "High Protein Herzhaftes Waffelsandwich mit Thunfisch-Creme", "slug": "high-protein-herzhaftes-waffelsandwich-mit-thunfisch-creme", "image": null, "recipe_yield": null, "total_time": null, "prep_time": "30", "cook_time": "0", "perform_time": null, "description": "", "recipe_category": [], "tags": [], "tools": [], "rating": null, "org_url": null, "recipe_ingredient": [], "date_added": null, "date_updated": null, "recipe_instructions": [], "nutrition": {"calories": "477", "fat_content": "17", "protein_content": "53", "carbohydrate_content": "25", "fiber_content": null, "sodium_content": null, "sugar_content": null}, "settings": {"public": false, "show_nutrition": false, "show_assets": false, "landscape_view": false, "disable_comments": true, "disable_amount": true, "locked": false}, "assets": [], "notes": [{"title": "author comment", "text": "Protein-Geheimwaffe(l)

# Add Image

In [17]:
image_url = recipe_api.get_image_url(api_key)
json_image_url = json.dumps({"url": image_url})
scrape_image_path = f"/recipes/{slug}/image"
r = client.post(scrape_image_path, data=json_image_url)
r.raise_for_status()

# Update Recipe Attributes

In [18]:
recipe_detail_path = f"/recipes/{slug}"

## Get Recipe Details

Need to fetch user and group id to be able to update receipt.

In [19]:
self_path = "/users/self"
r = client.get(self_path)
r.raise_for_status()
self_data = r.json()
recipe.user_id, recipe.group_id = self_data["id"], self_data["groupId"]

## Get Recipe ID

In [20]:
r = client.get(recipe_detail_path)
r.raise_for_status()
recipe_details = r.json()
recipe.id, recipe.user_id, recipe.group_id = recipe_details["id"], recipe_details["userId"], recipe_details["groupId"]

## Update Recipe

In [22]:
recipe_detail_path = f"/recipes/{slug}"
r = client.put(recipe_detail_path, data=recipe.json())
r.raise_for_status()